# Procedure when data added ; Start from "pre_inbody." [HERE]

## Can use "[RESET/RESTART]" because recalculated data is stored in pickle
1. Move new data from "C:\Users\bhuns\OneDrive\___Health Data\__DD studies\InBody CSV\ib97" into "JL_1/data/ib97"
2. GO to "pre_inbody"  and Hit >> [RESET/RESTART] to erase old results, reset kernal and run the cell. [Results in pickle]
3. GO to "nrmlz_data_dict"  and Hit >> [RESET/RESTART] to erase old results, reset kernal and run the cell. [Results in pickle]
4. GO to "plot_sandbox"  and Hit >> [RESET/RESTART] to erase old results, reset kernal and run the cell. [Results in graphs]
5. 

In [1]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")



/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


# Import

In [99]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
#from sys_funcs import 

# DEF functions for this workbook

In [100]:
# name the list in the call and this function will:
# ask for a substring and make a list of names that contain it.
def filter_column_names_interactive(df_col_nms):
    """
    Prompts for a substring and prints matching column names from df_col_nms.
    Assumes df_col_nms is a single-column DataFrame of column names.
    """
    substring = input("Enter substring to filter column names: ").strip()
    matches = df_col_nms[df_col_nms.iloc[:, 0].str.contains(substring, case=False, na=False)]
    
    if matches.empty:
        print(f"No column names contain '{substring}'.")
    else:
        print(f"Column names containing '{substring}':")
        for name in matches.iloc[:, 0]:
            print(f"  - {name}")


In [101]:
# this def function takes raw imported a data_dict cleans up the '14. Test Date / Time' then` This 13 for the 770
# attachs a "Cleaned_Timestamp" and a "dtv" 
import numpy as np
import pandas as pd
from datetime import datetime

In [102]:
def clean_data_dict(data_dict):
    cleaned_dict = data_dict.copy()
    base_date = datetime(1900, 1, 1)

    def safe_parse(ts):
        try:
            return (datetime.strptime(ts, "%Y%m%d%H%M%S") - base_date).days
        except Exception as e:
            print(f"Failed to parse timestamp {ts}: {e}")
            return np.nan

    timestamps = data_dict.get("13. Test Date / Time", [])
    cleaned_dict["Cleaned_Timestamp"] = [
        str(ts).strip()[:14] if str(ts).strip().lower() != "nan" and len(str(ts).strip()) >= 14 else np.nan
        for ts in timestamps
    ]
    cleaned_dict["dtv"] = [
        safe_parse(ts) if isinstance(ts, str) and len(ts) == 14 else np.nan
        for ts in cleaned_dict["Cleaned_Timestamp"]
    ]
    return cleaned_dict


# Create **ib970** by Loading **970.csv** files

In [103]:
print("091725")
ib970_a = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97",
    pattern="091725*",
    df_name="ib970_a"
)
print("091725",ib970_a)
print(ib970_a.columns)
      
ib970_b = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97",
    pattern="251201*",
    df_name="ib970_b"
)
print("251201",ib970_b)

# Combine them (assuming they return DataFrames)
import pandas as pd
ib970 = pd.concat([ib970_a, ib970_b], ignore_index=True)



091725
✅ Loaded 091725_bdailyn_20250922225826.csv with ISO-8859-1
✅ [ib970_a] Final DataFrame: 1 rows from 1 files.
💾 Saved to pickle: /home/bhuns/JL_2/ib970_a.pkl
091725      1. Name    2. ID   3. Height 4. Date of Birth 5. Gender  6. Age  \
0  <bdailyn>  bdailyn  6ft 06.0in      1939.02.23.         M    86.0   

  7. Mobile Number 8. Phone Number 9. Zip Code 10. Address  ...  \
0                -               -           -           -  ...   

  244. 50kHz-Whole Body Phase Angle_Z score 245. TBW/WT_T Score  \
0                                      -2.6                -1.4   

  246. TBW/WT_Z Score  247. SMI(SMM/Wt)_T score  248. SMI(SMM/Wt)_Z score  \
0                -0.3                      -1.9                      -0.6   

   249. ECM/BCM_T Score  250. ECM/BCM Z Score  Unnamed: 250  \
0                   8.0                   4.6           NaN   

                         source_file  encoding_used  
0  091725_bdailyn_20250922225826.csv     ISO-8859-1  

[1 rows x 253 columns]


# Create **ib970cln** *and* **df_ib970cln_col_nms** from **ib970**

## Cleanup the **'13. Test Date / Time'** so **datetime** will work and add cols ie. keys:
1. **Cleaned_Timestamp**
2. **dtv**

In [104]:
from collections.abc import Mapping
import pandas as pd

def kind_of(var):
    if isinstance(var, pd.DataFrame):
        return "dataframe"
    if isinstance(var, Mapping):   # covers dict, OrderedDict, defaultdict, etc.
        return "dict-like"
    return "other"

# Usage

print(kind_of(pd.DataFrame({"ib970cln ":[1,2]})))  # dataframe
print(kind_of({"ib970cln ": 1}))                   # dict-like
print(kind_of([1,2,3]))                            # other


dataframe
dict-like
other


In [105]:
kind_of(ib970)

'dataframe'

In [106]:
# verify 
# 
ib970    # all rows but not in chronological order

,1. Name,2. ID,3. Height,4. Date of Birth,5. Gender,6. Age,7. Mobile Number,8. Phone Number,9. Zip Code,10. Address,...,244. 50kHz-Whole Body Phase Angle_Z score,245. TBW/WT_T Score,246. TBW/WT_Z Score,247. SMI(SMM/Wt)_T score,248. SMI(SMM/Wt)_Z score,249. ECM/BCM_T Score,250. ECM/BCM Z Score,Unnamed: 250,source_file,encoding_used
0,<bdailyn>,bdailyn,6ft 06.0in,1939.02.23.,M,86.0,-,-,-,-,...,-2.6,-1.4,-0.3,-1.9,-0.6,8.0,4.6,NaN,091725_bdailyn_20250922225826.csv,ISO-8859-1


In [107]:
# this calls the def clean_data_dict () to clean bad testdate data and add "dtv" & "Cleaned_Timestamp" NANs still included
ib970cln = clean_data_dict(ib970)
print("ib970cln has been created with dtv & Cleaned_Timestamp cols added ")

ValueError: Length of values (0) does not match length of index (1)

In [ ]:
# verify still not in temporal order yet
#
ib970cln       #this has all the rows but not in chronological oreder

### Read the **df_ib970cln_col_nms** *from* **ib970cln**

In [ ]:
df_ib970cln_col_nms = pd.DataFrame(ib970cln.columns)
print("df_ib970cln_col_nms has been created ")
# df_ib970cln_col_nms

In [ ]:
# verify
# 
df_ib970cln_col_nms

## Save **ib970cln** and **df_ib970cln_col_nms** to **pickle**

In [ ]:
# Save ib970cln to a file
import pickle
with open("ib970cln.pkl", "wb") as f:
    pickle.dump(ib970cln, f)
print(" ib970cln is saved to pickle")


In [ ]:
# verify 
# 
ib970cln    # all rows but not in chronological order

In [ ]:
# verify 
# 
ib970cln_dtstmp = ib970cln[["dtv", "Cleaned_Timestamp"]]    # time and datestamp df

In [ ]:
# verify 
# 
ib970cln_dtstmp    #["ib970cln_dtstmp]

In [ ]:
# Save df_ib970cln_col_nms to a file
import pickle
with open("df_ib970cln_col_nms.pkl", "wb") as f:
    pickle.dump( df_ib970cln_col_nms, f)
print("  df_ib970cln_col_nms is saved to pickle")

In [ ]:
# Save ib970cln_dtstmp to a file
import pickle
with open("ib970cln_dtstmp.pkl", "wb") as f:
    pickle.dump( ib970cln_dtstmp, f)
print("  ib970cln_dtstmp is saved to pickle")

In [ ]:
# verify
# ib970mrn_grp1_nms

In [ ]:
# verify
# 
ib970cln_dtstmp

In [ ]:
print(list(ib970cln_dtstmp['Cleaned_Timestamp'].sort_values()))

# Creating  **ib970mrn_grp_dict** from **df_ib970cln_col_nms**

In [ ]:
print(list(ib970cln['Cleaned_Timestamp'].sort_values()))


##  **ib970mrn_grp1** from **ib970cln**

### Procedure

#### 
1. used to specify the keys[data_cols] needed in **grp1**
2. The are selected from **df_ib970cln_col_nms**
3. They are chosen manually by copying the names from **df_ib970cln_col_nms list** and pasting them into **ib970mrn_grp1** specification
4. To aid the a subscript from the input reduces the number of cos in the list.

#### 1. ib970mrn_grp all have keys= "dtv" and "Cleaned_Timestamp" + a different set of key combinations. [a key is the name of data col]
2. The ib970mrn_grp#_dict {key: grp# .........}
3. Each grp# starts with the "dtv" & "Cleaned_Timestamp" cols ready to have more cols added to the list.
4. A filter that shows all the "df_ib970cln_col_nms" that contain str supplied by input functions.
5. Chosen "col names" are copied into the list for that grp#
6. The grp# is stored via pickel and is read from pick either to use or to edit.
7. When editing, if a grp# does not exist the templet with the 2 cols is profide to star a new group.

### Implementation

#### The following readings are not working
184. Subcutaneous Fat
188. Visceral Fat
195. Abdominal Fat
196. V/S Ratio(Visceral Fat Area/Subcutaneous Fat Area ratio)
197. SFA(Subcutaneous Fat Area)
208. 50khz-Ab Impedance
209. 250khz-Ab Impedance


In [108]:
# make list of data_col names that have subsript ie THIS IS AN AID TO FIND THE RIGHT DA_COLS
df_col_nms = df_ib970cln_col_nms
df_col_nms
#filter_column_names_interactive(df_col_nms)
# filter_column_names_interactive(df_col_nms)
# link for ECM&BCM   https://copilot.microsoft.com/shares/J9Fbyou7dZXqDwmDD78S7

,0
0,1. Name
1,2. ID
2,3. Height
3,4. Date of Birth
4,5. Age
...,...
149,Unnamed: 149
150,source_file
151,encoding_used
152,Cleaned_Timestamp


In [ ]:
# for verification
# 
ib970mrn_grp1_nms

### Creating  **ib970mrn_grp1** from **ib970cln**

###  Overview

#### Extracts selected columns from a dict, validates timestamps, sorts chronologically,
    and filters by time-of-day window.

    Parameters:
    - source_dict: dict — your working dict (e.g. ib970cln)
    - column_group: list — list of column names to extract
    - timestamp_col: str — name of the timestamp column
    - start_time: datetime.time — lower bound for time-of-day filter
    - end_time: datetime.time — upper bound for time-of-day filter

    Returns:
    - df_filtered: pd.DataFrame — cleaned, sorted, and time-window-filtered DataFrame
    - df_errors: pd.DataFrame — rows with invalid timestamps
    """

###  Steps going from ib970cln to ib970cl_mrn_grp1
  

#### 
    # Step 1: Extract selected columns
    df = pd.DataFrame({col: source_dict[col] for col in column_group if col in source_dict})

    # Step 2: Validate timestamp column
    if timestamp_col not in df.columns:
        raise ValueError(f"Timestamp column '{timestamp_col}' not found in selected group.")

    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors="coerce")

    # Step 3: Split valid and invalid timestamps
    df_valid = df[df[timestamp_col].notna()].copy()
    df_errors = df[df[timestamp_col].isna()].copy()

    # Step 4: Sort chronologically
    df_valid = df_valid.sort_values(timestamp_col).reset_index(drop=True)

    # Step 5: Extract time component
    df_valid["time_only"] = df_valid[timestamp_col].dt.time

    # Step 6: Filter by time-of-day window
    df_filtered = df_valid[
        (df_valid["time_only"] >= start_time) &
        (df_valid["time_only"] <= end_time)
    ].copy()

    # Step 7: Drop helper column
    df_filtered.drop(columns=["time_only"], inplace=True)

    return df_filtered, df_errors

### Calling functions to implement the evolution from **ib970cln** to  **ib970cl_mrn_grp1** using the **ib970mrn_grp1_nms**
1. def extract_and_filter_by_time_window(**ib970cln**, **ib970mrn_grp1_nms**)
2. from **sys_funcs.py**

In [ ]:
#verify 
# ib970cln

In [ ]:
# verify 
# 
ib970mrn_grp1_nms

In [ ]:
# This function uses THIS FUNCTION stored in "sys_funcS.py to use the the ib970mrn_grp1_nms to collect the right data_cols from ib970cln for ib970mrn_grp1
#  IT THEN PERFORMS THE 7 STEPS TO RESULT IN       "ib970mrn_grp1 
column_group = ib970mrn_grp1_nms
# no_ no_no_no_no_The following will take morning an eve reading Put # in front below to to do only mnr
# no_no_no_no_no_no_bring   to this line to do only mrn reads column_group = df_ib970cln_col_nms
#
source_dict = ib970cln
# select the times to get mrn or all
# ib970mrn_grp1,df_errors = extract_and_filter_by_time_window(source_dict, column_group, timestamp_col="Cleaned_Timestamp",start_time=time(4, 0), end_time=time(11, 0))  #mrn
ib970mrn_grp1,df_errors = extract_and_filter_by_time_window(source_dict, column_group, timestamp_col="Cleaned_Timestamp",start_time=time(0, 0), end_time=time(23,59))    #all

In [ ]:
# verify
#
# ib970mrn_grp1 #["dtv"] #this is the final result "ib970mrn_grp1"`
# ib970cln #["160. 50kHz-Whole Body Phase Angle"]

In [ ]:
# verify to see the rejected teststamps
# df_errors

#### ib970mrn_grp1["14. Weight"]    
pd.set_option('display.max_rows', None)
ib970mrn_grp1[['dtv', 'Cleaned_Timestamp','14. Weight']]

In [ ]:
kind_of(ib970mrn_grp1)

In [ ]:
# Save ib970mrn_grp1 to a file
with open("ib970mrn_grp1.pkl", "wb") as f:
    pickle.dump(ib970mrn_grp1, f)
print(" ib970mrn_grp1 is saved to pickle")

In [ ]:
# verify
# ib970mrn_grp1 

In [ ]:
# Save ib970mrn_grp1_nms to a file
with open("ib970mrn_grp1_nms.pkl", "wb") as f:
    pickle.dump(ib970mrn_grp1_nms, f)
print(" ib970mrn_grp1_nms is saved to pickle")

# The rest are spares and duplicates >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [ ]:
print("all cells run ip970mrn_grp1 in pickle")